<a href="https://colab.research.google.com/github/AftabUdaipurwala/MLProjects/blob/main/Mahindra_Manufacturing_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Import Python Libraries***

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from google.colab import drive
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
from scipy import stats


# ***Mount Google Drive to Load Data***



In [ ]:
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/MyDrive/MahindraManufacturingAnalytics' # listing all the contents in the drive

# ***Load Data***

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MahindraManufacturingAnalytics/ManufacturingAnalytics.csv')
df.head()

In [ ]:
df= pd.DataFrame(df, index = range(0,9999))
df.head()

# ***Read and Understand Data***

In [ ]:
df.shape # checking shape of the data

In [ ]:
df.describe()
#''' Here we can clearly see that there are lot of outliers in each of thte variable,
# eg just check process temp, its min is 0, max is 11000 but 25th n 75th percentile is very narrow 308-311'''

In [ ]:
df.isnull().sum(axis = 0) ## Checking which all columns have null values. Here Couple of points to be noted are 1. the failure type missing values are 4 all togather

In [ ]:
df1=df[df.isnull().any(axis='columns')] # If we take subset of only missing values rows and then investigate
df1.shape

In [ ]:
df1.head(2)

In [ ]:
pd.crosstab(df1['Machine failure'],df1['RNF']) 
# So here we can safely say if we remove all these missing values it wont have any big impact on the y variable as only 1 of 50 values is a downtime record. 

In [ ]:
df.plot.hist(subplots=True, legend=True, layout=(4, 4)) # checking distribution of each variable

In [ ]:
# checking if we have any unique values in each variable or is everything only a single point data?
df2 = df.dropna(axis=0)
df2= df2.drop(['ID'],axis=1)
col_names = df2.columns

for i in col_names:
  x = len(df2[i].unique())
  print('Column Name is',i,'& Count of Unique values is ',x)

In [ ]:
df2.shape

In [ ]:
# creating a function to check values between 2 quantiles

def check_range(df, col):
  q_low = df[col].quantile(0.25)
  q_hi  = df[col].quantile(0.75)    
  print(col,"qhi",q_hi,'qlow',q_low)


In [ ]:
df3 = df2.drop(['Type','Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)
col_names=df3.columns
col_names

for i in col_names:
  check_range(df3,i)

In [ ]:
# creating a function to check values between 2 quantiles

def drop_numerical_outliers(df, col):
  q_low = df[col].quantile(0.01)
  q_hi  = df[col].quantile(0.99)    
  df= df[(df[col] < q_hi) & (df[col] > q_low)]
  return df


In [ ]:
df4= df2.copy()
df4 = drop_numerical_outliers(df4,'Air temperature [K]')
df4 = drop_numerical_outliers(df4,'Process temperature [K]')
df4 = drop_numerical_outliers(df4,'Rotational speed [rpm]')
df4 = drop_numerical_outliers(df4,'Torque [Nm]')
df4 = drop_numerical_outliers(df4,'Tool wear [min]')

In [ ]:
df4.shape

In [ ]:
df4.head()

In [ ]:
df4.describe()

In [ ]:
df5=df4[df4.isnull().any(axis='columns')] # If we take subset of only missing values rows and then investigate
df5.shape

In [ ]:
pd.crosstab(df4['Machine failure'],df4['TWF'])  # Here we can see that if we try to remove outliers a lot of downtime data is lost

In [ ]:
df4.plot.hist(subplots=True, legend=True, layout=(4, 4)) # checking distribution of each variable